<a href="https://colab.research.google.com/github/mamud1977/bootup/blob/main/NN_Recurrent_Stock%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [3]:
data = pd.read_csv('/content/sample_data/GOOG.csv')

data.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700


In [4]:
data.max(), data.min()

(Date          2019-10-04
 Open              1274.0
 High          1289.27002
 Low          1266.295044
 Close        1287.579956
 Adj Close    1287.579956
 Volume          82541500
 dtype: object,
 Date         2004-08-19
 Open          49.409801
 High          50.680038
 Low           47.800831
 Close         49.818268
 Adj Close     49.818268
 Volume             7900
 dtype: object)

In [5]:
data[data['Date']> '2015-01-01']

,Date,Open,High,Low,Close,Adj Close,Volume
2611,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447500
2612,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059800
2613,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899900
2614,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065000
2615,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353500
...,...,...,...,...,...,...,...
3804,2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1219.000000,1404100
3805,2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1205.099976,1273500
3806,2019-10-02,1196.979980,1196.979980,1171.290039,1176.630005,1176.630005,1615100
3807,2019-10-03,1180.000000,1189.060059,1162.430054,1187.829956,1187.829956,1621200


In [6]:
train_data= data[data['Date'] < '2015-01-01']
test_data = data[data['Date'] >= '2015-01-01']

train_data.shape   ## (2611,7)


(2611, 7)

In [7]:
test_data.shape  ## (1198,7)

(1198, 7)

In [8]:
train_data.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700


In [9]:
train_data.drop(['Date','Adj Close'], axis=1, inplace=True)

<ipython-input-9-a83f4273c27b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(['Date','Adj Close'], axis=1, inplace=True)


In [ ]:
train_data.head(4)

,Open,High,Low,Close,Volume
0,49.813286,51.835709,47.800831,49.982655,44871300
1,50.316402,54.336334,50.062355,53.952770,22942800
2,55.168217,56.528118,54.321388,54.495735,18342800
3,55.412300,55.591629,51.591621,52.239193,15319700


In [10]:
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
train_data[2:3]


array([[0.01025177, 0.01041574, 0.01166001, 0.00838273, 0.22215074]])

In [11]:
train_data.shape[1]

5

In [12]:
X_train = []
Y_train = []

for i in range(60, train_data.shape[0]):
  X_train.append(train_data[i-60:i])
  Y_train.append(train_data[i,0])

In [23]:
X_train[0].shape

(60, 5)

In [25]:
Y_train[0:1]

[0.07630296751038479]

In [26]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(units=60, activation='relu',  return_sequences=True, input_shape=(60,5)))

model.add(tf.keras.layers.LSTM(units=80, activation='relu', return_sequences=True))

model.add(tf.keras.layers.LSTM(units=100, activation='relu', return_sequences=True))

model.add(tf.keras.layers.Dense(units=1))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 60)              │          15,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 60, 80)              │          45,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 60, 100)             │          72,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 60, 1)               │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,461 (521.33 KB)

 Trainable params: 133,461 (521.33 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [29]:
model.fit(np.array(X_train), np.array(Y_train), epochs=10)

Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 15s 120ms/step - loss: 0.1060
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - loss: 0.0546
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 124ms/step - loss: 0.0520
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - loss: 0.0522
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - loss: 0.0481
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - loss: 0.0489
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 125ms/step - loss: 0.0502
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 116ms/step - loss: 0.0515
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - loss: 0.0476
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - loss: 0.0500
